In [51]:
###########Script to extract data from TSV files, load into MySQL database and query database to output CSV files.###########

In [52]:
import requests
import os
import gzip
from tqdm.notebook import trange

#Importer les données IMDB
# liste des urls pour chaque fichier à telecharger
urls = ['https://datasets.imdbws.com/name.basics.tsv.gz',
       'https://datasets.imdbws.com/title.basics.tsv.gz', 
       'https://datasets.imdbws.com/title.episode.tsv.gz', 
       'https://datasets.imdbws.com/title.principals.tsv.gz', 
       'https://datasets.imdbws.com/title.ratings.tsv.gz']

# telechargement des fichiers
# for url in urls:
#     filename = url.split('/')[-1]
#     target_path = os.path.join("..", "data", "raw", filename)

#     response = requests.get(url, stream=True)  
   
#     if response.status_code == 200:
#         with open(target_path, 'wb') as f:
#             f.write(response.raw.read())
#     tsv_path = target_path.replace(".gz", "")

#     with gzip.open(target_path, 'rb') as gz_file:
#         with open(tsv_path, 'wb') as f:
#             f.write(gz_file.read())
            
#     for i in trange(1, desc='Statut'):
#         print('Fichier téléchargé :', tsv_path)
   

In [53]:
import subprocess

# def run_command(command, shell=True):
#     """Helper to run shell or list-based commands with error handling."""
#     try:
#         result = subprocess.run(command, shell=shell, check=True, capture_output=True, text=False)
#         print(result.stdout)
#     except subprocess.CalledProcessError as e:
#         print(f"Error running command: {command}")
#         print(e.stderr)



In [ ]:
# Demarrer le conteneur Docker pour MySQL
print("🚀 Starting Docker container...")
COMPOSE_FILE="../docker/docker-compose.yml"
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


In [81]:
#change filename for each query
#copy the SQL file into the container
file_name = "marvel_directors.sql"
output_name = "movie_directors.csv"  # filename for the exported CSV

print("📁 Copying sql file into docker container...")
result = subprocess.run(["docker", "cp", f"../mysql/{file_name}", f"moviesdb_mysql:/tmp/{file_name}"], capture_output=True, text=True, check=False)
print(result.stderr)

📁 Copying sql file into docker container...



In [82]:
# Run the SQL query inside the container and save output to a CSV file
from pathlib import Path
CONTAINER = "moviesdb_mysql"
SQL_IN_CONTAINER = f"/tmp/{file_name}"  # path inside the container where the SQL file is located
DB = "IMDb"  # database name inside the MySQL container
HOST_OUT_PATH = Path(f"../data/tests/{output_name}")
container_out = f"/tmp/{output_name}"  # full path inside the container for the output CSV

mysql_cmd = f"mysql -u root -proot {DB}"

mysql_cmd += f' -e "source {SQL_IN_CONTAINER}" > {container_out}'  # execute SQL file inside mysql and redirect stdout to CSV path

cmd = ["docker", "exec", "-i", CONTAINER , "sh" , "-c",mysql_cmd]  # docker exec command to run the mysql command inside the container
# run_command(cmd)  # run the docker exec command to produce the CSV inside the container

with HOST_OUT_PATH.open("wb") as f:
    subprocess.run(cmd, check=True, stdout=f)

# copy the resulting CSV from container to host
subprocess.run(["docker", "cp", f"{CONTAINER}:{container_out}", f"../data/tests/{output_name}"],check=True)  # copy the CSV from container to host path


mysql: [Warning] Using a password on the command line interface can be insecure.


CompletedProcess(args=['docker', 'cp', 'moviesdb_mysql:/tmp/movie_directors.csv', '../data/tests/movie_directors.csv'], returncode=0)

In [83]:
#read csv into pandas dataframe
import pandas as pd


orig_path = "../data/tests/" + output_name

# read raw bytes and try common encodings, fallback to latin-1 with replacement
# with open(orig_path, 'rb') as f:
#     raw = f.read()
#handle malformed json in character_name column by extracting first character from JSON array
import json
def extract_first_character(x):
    """Extract first character from JSON array, handle JSON decode errors gracefully"""
    if pd.notnull(x) and x != "\\N" and str(x).strip():
        try:
            parsed = json.loads(x)
            return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else x
        except (json.JSONDecodeError, ValueError, TypeError):
            # If JSON parsing fails, return the value as-is
            return x
    return None

marvel_df = pd.read_csv(orig_path, sep='\t', encoding='latin-1')
if "characters" in output_name:
    
    marvel_df["character_name"] = marvel_df["character_name"].apply(extract_first_character)
    # marvel_df["character_name"] = marvel_df["character_name"].str.replace(r"[\[\]'\"]", "", regex=True)


print(marvel_df.head())
print(marvel_df.shape)
print("Returned rows:", len(marvel_df))
marvel_df.to_csv("../data/tests/"+output_name, index=False)


    title_id  person_id  person_name       job  category
0  tt0371746  nm0032696     Avi Arad  producer  producer
1  tt0371746  nm0270559  Kevin Feige  producer  producer
2  tt1228705  nm0270559  Kevin Feige  producer  producer
3  tt0458339  nm0270559  Kevin Feige  producer  producer
4  tt0800369  nm0270559  Kevin Feige  producer  producer
(108, 5)
Returned rows: 108


In [58]:
# # read csv into pandas dataframe
# import pandas as pd
# marvel_characters_df = pd.read_csv("../data/tests/marvel_characters.csv")

# # character_name is formatted as a string representation of a list, convert it back to list and get first element
# import ast

# marvel_characters_df['character_name'] = marvel_characters_df['character_name'].str.replace(r"\s*\([^)]*\)", "", regex=True)

# marvel_characters_df['character_name'] = marvel_characters_df['character_name'].apply(
#     lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else None
# )

# marvel_characters_df.head()

In [ ]:
# #Stop and remove the container

print("🚀 Stopping Docker container")
COMPOSE_FILE="../docker/docker-compose.yml"
CONTAINER_NAME="moviesdb_mysql"

subprocess.run(["docker", "stop", CONTAINER_NAME])
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "down"])